#### Named Entity Recognition

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string # special operations on strings
import spacy # language models

from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

In [6]:
import pandas
book=pd.read_csv("apple.txt",sep=";",error_bad_lines=False)

In [20]:
book[20]

"Apple,apple,apple, thats the first thing that must have flashed in you mind when you saw this product, so here i have a total breif review on the laptop...***Pros***4k Video Editing with Final Cut Pro X on this thing works like butter, there is heating issue that i am going to go to apple service center to get it checkedEveryday works like checking mail and is just gr8The Battery is just OVERPOWERED, works for around 11-12 hours on continuous work and 8-9 hours when editingThe graphics card scores 30 fps in cinebench which is not that badThe performance is also fine, 280 cb processor score on cinebenchSpeakers are really good with perfect bassDisplay brightness is 300 nits, which is fineThe Wi-Fi is extremely good, can get a network even from the living room while the router is in the bedroomVery thin and lightweightMacOS is good for School work if you know all the office applications for mac, for eg.Pages is WordMagSafe 2 is the best partGood Trackpad and Keyboard with backlightThe S

In [7]:
book = [x.strip() for x in book.x] # remove both the leading and the trailing characters
book = [x for x in book if x] # removes empty strings, because they are considered in Python as False
book[0:10]

["It's amazing..... battery last around 10 -12 hrs.....best laptop for students and coders",
 "Finally a MacBook possession. Got it in 50k in Diwali sale. Technically good laptop.Good News.luxury product, niche technology, good battery life, reliable machine (other laptops crash and become defunct in 6-10 years), no anti-virus required, security of data and transactions much better. Terrific sense of possession & pride.Bad News. Problems will be there if you are switching from windows based system - very less space in hard disk, no CD drive, inability to transfer data from Mac to your existing external hard disc unless you format it, apps are mostly paid and re unreasonably expensive (no free apps which are available otherwise on google play store, even the angry bird costs Rs 400 !!), all printers are not compatible (e.g. the most economical MFD laser printer RICOH SP 111 can't be used), huge compatibility issues with Pages (MS Word) and Keynote (powerpoint) unless you master it by wo

In [10]:
##Part Of Speech Tagging
nlp = spacy.load('en_core_web_md') 

one_block = book[20]
doc_block = nlp(one_block)
spacy.displacy.render(doc_block, style='ent', jupyter=True)

In [12]:
one_block

"Apple,apple,apple, thats the first thing that must have flashed in you mind when you saw this product, so here i have a total breif review on the laptop...***Pros***4k Video Editing with Final Cut Pro X on this thing works like butter, there is heating issue that i am going to go to apple service center to get it checkedEveryday works like checking mail and is just gr8The Battery is just OVERPOWERED, works for around 11-12 hours on continuous work and 8-9 hours when editingThe graphics card scores 30 fps in cinebench which is not that badThe performance is also fine, 280 cb processor score on cinebenchSpeakers are really good with perfect bassDisplay brightness is 300 nits, which is fineThe Wi-Fi is extremely good, can get a network even from the living room while the router is in the bedroomVery thin and lightweightMacOS is good for School work if you know all the office applications for mac, for eg.Pages is WordMagSafe 2 is the best partGood Trackpad and Keyboard with backlightThe S

In [11]:
for token in doc_block[0:20]:
    print(token, token.pos_)

Apple NOUN
, PUNCT
apple NOUN
, PUNCT
apple NOUN
, PUNCT
that DET
s VERB
the DET
first ADJ
thing NOUN
that DET
must AUX
have AUX
flashed VERB
in ADP
you PRON
mind NOUN
when ADV
you PRON


In [ ]:
#Filtering for nouns and verbs only
nouns_verbs = [token.text for token in doc if token.pos_ in ('NOUN', 'VERB')]
print(nouns_verbs[5:25])

In [ ]:
#Counting tokens again
cv = CountVectorizer()

X = vectorizer.fit_transform(nouns_verbs)
sum_words = X.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
wf_df = pd.DataFrame(words_freq)
wf_df.columns = ['word', 'count']

wf_df[0:10]

In [ ]:
##Visualizing results
#Barchart for top 10 nouns + verbs
wf_df[0:10].plot.bar(x='word', figsize=(12,8), title='Top verbs and nouns')

#### Emotion Mining

In [ ]:
#Sentiment analysis
afinn = pd.read_csv('Afinn.csv', sep=',', encoding='latin-1')
afinn.shape

In [ ]:
afinn.head()

In [ ]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [ ]:
affinity_scores = afinn.set_index('word')['value'].to_dict()
take(20, affinity_scores.items())

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string # special operations on strings
import spacy # language models

from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

import pandas
book=pd.read_csv("apple.txt",error_bad_lines=False)
book = [x.strip() for x in book.x] # remove both the leading and the trailing characters
book = [x for x in book if x] # removes empty strings, because they are considered in Python as False

In [ ]:
from nltk import tokenize
sentences = tokenize.sent_tokenize(" ".join(book))
sentences[5:15]

In [ ]:
sent_df = pd.DataFrame(sentences, columns=['sentence'])
sent_df

In [ ]:
#Custom function :score each word in a sentence in lemmatised form, 
#but calculate the score for the whole original sentence.
nlp = spacy.load('en')
sentiment_lexicon = affinity_scores

def calculate_sentiment(text: str = None) -> float:
    sent_score = 0
    if text:
        sentence = nlp(text)
        for word in sentence:
            sent_score += sentiment_lexicon.get(word.lemma_, 0)
    return sent_score

In [ ]:
# test that it works
calculate_sentiment(text = 'very sad')

In [ ]:
sent_df['sentiment_value'] = sent_df['sentence'].apply(calculate_sentiment)

In [ ]:
# how many words are in the sentence?
sent_df['word_count'] = sent_df['sentence'].str.split().apply(len)
sent_df['word_count'].head(10)

In [ ]:
sent_df.sort_values(by='sentiment_value').tail(10)

In [ ]:
# Sentiment score of the whole review
sent_df['sentiment_value'].describe()

In [ ]:
# Sentiment score of the whole review
sent_df[sent_df['sentiment_value']<=0].head()

In [ ]:
sent_df[sent_df['sentiment_value']>=20].head()

In [ ]:
sent_df['index']=range(0,len(sent_df))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(sent_df['sentiment_value'])

In [ ]:
plt.figure(figsize=(15, 10))
sns.lineplot(y='sentiment_value',x='index',data=sent_df)

In [ ]:
sent_df.plot.scatter(x='word_count', y='sentiment_value', figsize=(8,8), title='Sentence sentiment value to sentence word count')